# Import packages

In [ ]:
import os
import cv2
import cmaps
import cmocean
import numpy as np
import xarray as xr
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from netCDF4 import Dataset
from datetime import date
from mpl_toolkits.basemap import Basemap
from matplotlib.pyplot import Polygon
from matplotlib import rcParams
from matplotlib.backends.backend_pdf import PdfPages
version = mpl.__version__
rcParams['font.family'] = 'sans-serif'
directory   = '/srv/scratch/z3533156'

# Import time functions

In [ ]:
def datestring_to_serial_day(datestring,epochY=1990,epochm=1,epochd=1,epochH=0,epochM=0):
    import pandas as pd
    import datetime
    serial_day_timedelta = pd.to_datetime(datestring) - datetime.datetime(epochY,epochm,epochd,epochH,epochM)
    corrected_serial_day_number = serial_day_timedelta.days + serial_day_timedelta.seconds/86400
    return corrected_serial_day_number
def serial_day_to_datestring(day,epochY=1990,epochm=1,epochd=1,epochH=0,epochM=0):
    import datetime
    corrected_date = datetime.datetime(epochY,epochm,epochd,epochH,epochM) + datetime.timedelta(day)
    return corrected_date.strftime("%Y-%m")  

# Read data

In [ ]:
dataset        = sio.loadmat(directory+'/MHW/Figure_data/Figure3_ERA5_SLA.mat')
dataset1       = sio.loadmat(directory+'/MHW/Figure_data/Figure1_OSTIA_MHW.mat')
AVISO_UGOS     = Dataset(directory+'/MHW/AVISO/AVISO_EAC_monthly_UGOS.nc','r')
AVISO_VGOS     = Dataset(directory+'/MHW/AVISO/AVISO_EAC_monthly_VGOS.nc','r')
AVISO_KE       = Dataset(directory+'/MHW/AVISO/AVISO_EAC_monthly_KE.nc','r')
AVISO_SSH      = Dataset(directory+'/MHW/AVISO/AVISO_EAC_monthly_SSH.nc','r')
SSH_clim       = dataset1['SSH_monthly_clim']
ugos_monthly   = np.transpose(AVISO_UGOS.variables['ugos'][:,:,:],[2,1,0])
vgos_monthly   = np.transpose(AVISO_VGOS.variables['vgos'][:,:,:],[2,1,0])
ke_monthly     = np.transpose(AVISO_KE.variables['ke'][:,:,:],[2,1,0])
SSH_monthly    = np.transpose(AVISO_SSH.variables['SSH'],[2,1,0])
lon0           = dataset['lon_ERA5'][:,0]
lat0           = dataset['lat_ERA5'][:,0]
lon1           = dataset['lon_AVISO'][:,0]
lat1           = dataset['lat_AVISO'][:,0]
time_monthly   = dataset['time_monthly'][:,0]
SLA_monthly    = dataset['SLA_monthly']
msnhfa_monthly = dataset['msnhfa_monthly']
lon_ERA5       = np.tile(lon0,[np.size(lat0,0),1]).transpose()
lat_ERA5       = np.tile(lat0,[np.size(lon0,0),1])
lon_AVISO      = np.tile(lon1,[np.size(lat1,0),1]).transpose()
lat_AVISO      = np.tile(lat1,[np.size(lon1,0),1])
time           = datestring_to_serial_day("1993-01-01",epochY=1990,epochm=1,epochd=1,epochH=0,epochM=0)
monthly_t      = np.arange(date(1993,1,1).toordinal(),date(2022,12,31).toordinal()+1,365.5)
monthly_dates  = [date.fromordinal(tt.astype(int)) for tt in monthly_t]

# Plot the spatial distribution of net heat fluxes and v component anomalies

In [ ]:
fig    = plt.figure(figsize=(80, 60))
gs     = gridspec.GridSpec(3,3)
m      = Basemap(projection='merc',llcrnrlat=-40.0,urcrnrlat=-24+0.01,llcrnrlon=145.0,urcrnrlon=165,resolution='i')
x1, y1 = m(lon_ERA5,  lat_ERA5)
x2, y2 = m(lon_AVISO, lat_AVISO)
padspacescale = 50
labelpadscale = 15
linefont      = 12
labelfont     = 90
scale         = 1.5
levels1       = np.linspace(0,0.3,50)
levels2       = np.linspace(-100,100,50)
levels3       = np.linspace(-0.5,0.5,50)
tick_marks1   = np.linspace(0,0.3,4)
tick_marks2   = np.linspace(-100,100,5)
tick_marks3   = np.linspace(-0.5,0.5,5)
sla_levels1   = np.linspace(0,0.5,3)
sla_levels2   = np.linspace(-0.5,0,3)
sla_levels3   = np.linspace(-0.001,0.001,2)
ssh_levels1   = np.linspace(0.949,0.951,2)
cmaps1        = cmaps.cmocean_matter 
cmaps2        = cmaps.cmocean_balance
labels        = ['a','b','c','d','e','f','g','h','i']
for i in range(9):
    if i==0:
        j=11
    else:
        j=i-1
    ax = fig.add_subplot(gs[i])
    l, b, w, h = ax.get_position().bounds
    m      = Basemap(projection='merc',llcrnrlat=-40.0,urcrnrlat=-24+0.01,llcrnrlon=145.0,urcrnrlon=165,resolution='i')
    m.drawcoastlines(color='0.1',  linewidth=0.5*linefont)
    m.drawmapboundary(color='0.1', linewidth=0.5*linefont)
    m.fillcontinents(color='0.95', lake_color='white')
    m.drawmeridians(np.arange(140,165, 5),labels=[0,0,0,0],linewidth=0.4*linefont,dashes=[2,2],color='.7',fontsize=labelfont)        
    k      = np.mod(i,3)+347
    yy     = np.arange(0, y2.shape[0], 4)
    xx     = np.arange(0, x2.shape[1], 4)
    points = np.meshgrid(yy, xx)
    ugos   = ugos_monthly[:,:,k]
    vgos   = vgos_monthly[:,:,k]
    if np.mod(i,3)==0:
        plt.ylabel('Latitude',fontsize=1.2*labelfont,labelpad=5*padspacescale,family='sans-serif')
        m.drawparallels(np.arange(-40,-10, 2),labels=[1,0,0,0],linewidth=0.4*linefont,dashes=[2,2],color='.7',fontsize=labelfont)
    else:
        m.drawparallels(np.arange(-40,-10, 2),labels=[0,0,0,0],linewidth=0.4*linefont,dashes=[2,2],color='.7',fontsize=labelfont)
    if i<3:
        CB1 = m.contourf(x2, y2, ke_monthly[:,:,k],cmap=cmaps1,levels=levels1,origin='lower',extend='both')
        CS1 = m.contour(x2, y2,  SSH_monthly[:,:,k],ssh_levels1,linewidths=2*linefont,linestyles='solid',colors='black')
        CS2 = m.contour(x2, y2,  SSH_clim[:,:,j], ssh_levels1,linewidths=2*linefont,linestyles='solid',colors='gray')
        Q   = m.quiver(x2[tuple(points)],y2[tuple(points)],ugos[tuple(points)],vgos[tuple(points)],color='gray',width=0.005,scale=6)
        qk1 = plt.quiverkey(Q,0.14,0.68, 0.5, r' ',labelpos='N',color='gray',linewidth=linefont,fontproperties={'size': labelfont})
        qk2 = plt.quiverkey(Q,0.145,0.695,0.5, r'0.5 m$^{-1}$',labelpos='N',color='None',linewidth=linefont,fontproperties={'size':0.9*labelfont})
    elif i<6:
        b   = b-0.1
        CB2 = m.contourf(x1, y1, msnhfa_monthly[:,:,k],cmap=cmaps2,levels=levels2,origin='lower',extend='both') 
        CS2 = m.contour(x2, y2,  SSH_monthly[:,:,k],ssh_levels1,linewidths=2*linefont,linestyles='solid',colors='black')
        lon2    = np.array([lon_AVISO[22,20],lon_AVISO[25,20],lon_AVISO[33,33],lon_AVISO[30,33],lon_AVISO[22,20]])
        lat2    = np.array([lat_AVISO[22,20],lat_AVISO[25,20],lat_AVISO[33,33],lat_AVISO[30,33],lat_AVISO[22,20]])        
        x22,y22 = m(lon2,lat2)
        m.plot(x22,  y22,  linewidth=labelpadscale, linestyle='solid', color='green')
    else:
        b   = b-0.2
        plt.xlabel('Longitude',fontsize=1.2*labelfont,labelpad=2*padspacescale,family='sans-serif')
        m.drawmeridians(np.arange(140,165, 5),labels=[0,0,0,1],linewidth=0.4*linefont,dashes=[2,2],color='.7',fontsize=labelfont)        
        CB3 = m.contourf(x2, y2, SLA_monthly[:,:,k],cmap=cmaps2,levels=levels3,origin='lower',extend='both')   
        CS1 = m.contour(x2,  y2, SLA_monthly[:,:,k],sla_levels1,linewidths=0.8*linefont,linestyles='solid',colors='black')
        CS2 = m.contour(x2,  y2, SLA_monthly[:,:,k],sla_levels2,linewidths=0.8*linefont,linestyles='dashed',colors='black')
        CS3 = m.contour(x2,  y2, SLA_monthly[:,:,k],sla_levels3,linewidths=linefont,linestyles='solid',colors='gray')
    lon3    = np.array([lon_AVISO[20,8],lon_AVISO[61,8],lon_AVISO[61,41],lon_AVISO[20,41],lon_AVISO[20,8]])
    lat3    = np.array([lat_AVISO[20,8],lat_AVISO[61,8],lat_AVISO[61,41],lat_AVISO[20,41],lat_AVISO[20,8]])
    x22,y22 = m(lon3,lat3)
    cx11, cy11 = m(153.5, -33.2)
    cx22, cy22 = m(156.8, -34.5)  
    cx33, cy33 = m(151.5, -37)
    cx44, cy44 = m(153,   -35)
    if i==6:
        m.plot(x22,  y22,  linewidth=labelpadscale, linestyle='solid', color='xkcd:bright yellow')
        plt.text(cx11, cy11, 'CE1', color='xkcd:bright yellow', fontsize=1.0*labelfont, fontweight='bold')
        plt.text(cx22, cy22, 'CE2', color='xkcd:bright yellow', fontsize=1.0*labelfont, fontweight='bold') 
        plt.text(cx33, cy33, 'CE3', color='xkcd:bright yellow', fontsize=1.0*labelfont, fontweight='bold')
        plt.text(cx44, cy44, 'AE',  color='xkcd:bright yellow', fontsize=1.0*labelfont, fontweight='bold')
    ax.spines['bottom'].set_linewidth(labelpadscale)
    ax.spines['left'].set_linewidth(labelpadscale)
    ax.spines['top'].set_linewidth(labelpadscale)
    ax.spines['right'].set_linewidth(labelpadscale)
    cx1,cy1 = m(146.0, -25.35) 
    plt.text(cx1, cy1,serial_day_to_datestring(time+time_monthly[k]), color='dodgerblue', fontsize=1.2*labelfont)
    plt.title(labels[i], fontsize=1.2*labelfont,loc='left',pad=0.8*padspacescale, weight='bold',family='sans-serif')
    ax.set_position([l, b, scale*w, scale*h])
    plt.tick_params(axis='both',which='major',bottom='on',left='on',length=50.0,width=20,colors='black',direction='out')
cbaxes1 = fig.add_axes([0.98, 0.665, 0.015, 0.32])
cb1     = plt.colorbar(CB1,orientation='vertical',cax = cbaxes1)
cb1.set_ticks(tick_marks1)
cb1.set_label(r'Kinetic energy (m$^{2}$ s$^{-2}$)', fontsize=labelfont,labelpad=10)
cb1.ax.tick_params(labelsize=labelfont)
cbaxes2 = fig.add_axes([0.98, 0.29, 0.015, 0.32])
cb2     = plt.colorbar(CB2,orientation='vertical',cax = cbaxes2)
cb2.set_ticks(tick_marks2)
cb2.set_label(r'Heat fluxes anomalies (W m$^{-2}$)', fontsize=labelfont,labelpad=10)
cb2.ax.tick_params(labelsize=labelfont)
cbaxes3     = fig.add_axes([0.98, -0.075, 0.015, 0.32])
cb3 = plt.colorbar(CB3,orientation='vertical',cax = cbaxes3)
cb3.set_ticks(tick_marks3)
cb3.set_label(r'Sea level anomalies (m)', fontsize=labelfont,labelpad=10)
cb3.ax.tick_params(labelsize=labelfont)
fig.savefig(directory+'/MHW/Figure_plots/Figure3_KE_NHF_SLA.pdf',dpi=300,bbox_inches = 'tight') 